In [ ]:
%pip install comet_ml #libreria a instalar

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 586.4/586.4 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 59.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.8/252.8 kB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.3/54.3 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.2/512.2 kB 26.2 MB/s eta 0:00:00
  Attempting uninstall: websocket-client
    Found existing installation: websocket-client 1.6.4
    Uninstalling websocket-client-1.6.4:
      Successfully uninstalled websocket-client-1.6.4
  Attempting uninstall: python-box
    Found existing installation: python-box 7.1.1
    Uninstalling python-box-7.1.1:
      Successfully uninstalled python-box-7.1.1


In [ ]:
import comet_ml
comet_ml.init(project_name="RNA_Densa_L1") #nombre a dar al experimento

Please paste your Comet API key from https://www.comet.com/api/my/settings/
(api key may not show as you type)
Comet API key: ··········


COMET INFO: Valid Comet API Key saved in /root/.comet.config (set COMET_CONFIG to change where it is saved).


In [ ]:
# Imports
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import mnist #base de datos
from tensorflow.keras.models import Sequential # checar cada uno de los que faltan
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Input
from tensorflow.keras.optimizers import RMSprop, SGD, Adam
from tensorflow.keras import regularizers
from keras.callbacks import ModelCheckpoint # sirve para dar respuesta del proceso de entrenamiento de la red

In [ ]:
experiment = comet_ml.Experiment(  # para activar la Graficacion en comet
    auto_histogram_weight_logging=True,
    auto_histogram_gradient_logging=True,
    auto_histogram_activation_logging=True,
    log_code=True, # para subir el codigo a comet para no perder el progreso
)

COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/nicglowss/rna-densa-l1/00694a9f7b2c4c0bb4997420e97a0950
COMET INFO:   Uploads:
COMET INFO:     environment details : 1
COMET INFO:     filename            : 1
COMET INFO:     installed packages  : 1
COMET INFO:     notebook            : 2
COMET INFO:     os packages         : 1
COMET INFO:     source_code         : 1
COMET INFO: 
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Couldn't find a Git repository in '/content' nor in any parent directory. Set `COMET_GIT_DIRECTORY` if your Git Reposit

In [ ]:
# pre-procesamiento, aqui normalizamos la informacion de los pixeles
# Lo principal con la red es tener la misma normalizacion y tamaño de imagen.
dataset=mnist.load_data()# en el dataset se guardan todas las imagenes del mnist con sus etiquetas
(x_train, y_train), (x_test, y_test) = dataset

# se van a guardar en el forato de binario
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# =numero de pixeles normalizando el no. de pixeles
x_train /= 255  # x_trainv = x_trainv/255
x_test /= 255

#clasificacion de clases
num_classes=10
y_trainc = keras.utils.to_categorical(y_train, num_classes) #analiza los datos y los acomoda en las distintas clases
y_testc = keras.utils.to_categorical(y_test, num_classes)

11490434/11490434 [==============================] - 0s 0us/step


In [ ]:
# Parametros
parameters = {
    "batch_size": 10,
    "epochs": 30,
    "optimizer": "SGD",
    "loss": "categorical_crossentropy",
}

experiment.log_parameters(parameters) # diccionario subido a comet

In [ ]:
#Arquitetura de la red
model = Sequential()
# aqui empezamos
model.add(Input(shape=(28,28))) #Flaten no tiene la opcion input_shape por lo tanto se tiene que agregar esta capa
                                #28x28 es el tamaño de la imagen
model.add(Flatten()) #Otra forma de aplanar las imagenes, las forma en una linea
model.add(Dense(784, activation='sigmoid', kernel_regularizer=regularizers.L1(0.001)))
                                            # esta es la capa densa de no neuronas de forma vertical (en este caso agarramos el total de pixeles)
                                            # su funcion de activacion es la sigmoide (respuesta de la neurona de 0 a 1)
                                            # Estas dos son las principales

model.add(Dense(10, activation='softmax'))  # son las de salida literal porque estan al final

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 784)               615440    
                                                                 
 dense_1 (Dense)             (None, 10)                7850      
                                                                 
Total params: 623290 (2.38 MB)
Trainable params: 623290 (2.38 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
# specify the path where you want to save the model
filepath = "mejor-modelo1.1.hdf5"

# initialize the ModelCheckpoint callback
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

In [ ]:
# Inicia el entrenamiento de la red
model.compile(loss=parameters['loss'],optimizer='RMSp',metrics=['accuracy'])
historial=model.fit(x_train, y_trainc,
                    batch_size=parameters['batch_size'],
                    epochs=parameters["epochs"],
                    verbose=1,
                    validation_data=(x_test, y_testc),
                    callbacks=[checkpoint])

COMET INFO: Ignoring automatic log_parameter('verbose') because 'keras:verbose' is in COMET_LOGGING_PARAMETERS_IGNORE


Epoch 1/30
5994/6000 [============================>.] - ETA: 0s - loss: 2.1553 - accuracy: 0.7316
Epoch 1: val_loss improved from inf to 0.92737, saving model to mejor-modelo1.1.hdf5
6000/6000 [==============================] - 46s 8ms/step - loss: 2.1541 - accuracy: 0.7317 - val_loss: 0.9274 - val_accuracy: 0.8657
Epoch 2/30
   1/6000 [..............................] - ETA: 1:30 - loss: 1.0737 - accuracy: 0.8000

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


6000/6000 [==============================] - ETA: 0s - loss: 0.9971 - accuracy: 0.8326
Epoch 2: val_loss did not improve from 0.92737
6000/6000 [==============================] - 47s 8ms/step - loss: 0.9971 - accuracy: 0.8326 - val_loss: 1.3245 - val_accuracy: 0.7776
Epoch 3/30
5997/6000 [============================>.] - ETA: 0s - loss: 0.8658 - accuracy: 0.8569
Epoch 3: val_loss improved from 0.92737 to 0.89018, saving model to mejor-modelo1.1.hdf5
6000/6000 [==============================] - 44s 7ms/step - loss: 0.8658 - accuracy: 0.8569 - val_loss: 0.8902 - val_accuracy: 0.8601
Epoch 4/30
5997/6000 [============================>.] - ETA: 0s - loss: 0.7944 - accuracy: 0.8722
Epoch 4: val_loss improved from 0.89018 to 0.68027, saving model to mejor-modelo1.1.hdf5
6000/6000 [==============================] - 54s 9ms/step - loss: 0.7943 - accuracy: 0.8723 - val_loss: 0.6803 - val_accuracy: 0.9065
Epoch 5/30
5998/6000 [============================>.] - ETA: 0s - loss: 0.7477 - accuracy:

In [ ]:
experiment.log_model("MNIST1", "mejor-modelo1.1.hdf5")

{'web': 'https://www.comet.com/api/asset/download?assetId=2fbb7f3794004f06b0335111b7e1f8e6&experimentKey=bf25eae6a70e42ef97b4d6a21f675bd1',
 'api': 'https://www.comet.com/api/rest/v2/experiment/asset/get-asset?assetId=2fbb7f3794004f06b0335111b7e1f8e6&experimentKey=bf25eae6a70e42ef97b4d6a21f675bd1',
 'assetId': '2fbb7f3794004f06b0335111b7e1f8e6'}

In [ ]:
experiment.end()

COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/nicglowss/rna-densa-l1/bf25eae6a70e42ef97b4d6a21f675bd1
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     accuracy [30]                  : (0.7317166924476624, 0.9228833317756653)
COMET INFO:     batch_accuracy [18000]         : (0.0, 1.0)
COMET INFO:     batch_loss [18000]             : (0.3177264928817749, 24.334575653076172)
COMET INFO:     epoch_duration [30]            : (41.96843035100005, 58.20100335200004)
COMET INFO:     loss [30]                      : (0.5692181587219238, 2.154052734375)
COMET INFO:     val_accuracy [30]              : (0.7775999903678894, 0.939300000667572)
COMET INFO:     val_loss [30]      